# Game Engine

In [ ]:
# | default_exp game.engine

In [ ]:
# | hide
%load_ext lab_black

In [ ]:
# | export
import importlib
import os

All games are located within the `games` subdirectory.

## Engine

In [ ]:
# | export
TITLE_DIR = "engine/game/titles"
TITLE_MODULE = "rl18xx.game.engine.game.title"

In [ ]:
# | export


class Engine:
    def __init__(self):
        self.game_modules = self._import_game_modules()
        self.game_meta = self._collect_game_meta()
        self.games = self._load_games()

    def _import_game_modules(self):
        game_modules = {}
        for filename in os.listdir(TITLE_DIR):
            if filename.endswith(".ipynb"):
                module_name = filename[3:-6]
                module_path = f"{TITLE_MODULE}.{module_name}"
                game_modules[module_name] = importlib.import_module(module_path)
        return game_modules

    def _collect_game_meta(self):
        game_meta_by_title = {}
        for name, module in self.game_modules.items():
            if hasattr(module, "Meta"):
                meta = getattr(module, "Meta")
                game_meta_by_title[meta.title()] = meta
        return game_meta_by_title

    def _load_games(self):
        games = {}
        for title in self.game_meta:
            if hasattr(self.game_modules["g" + title], "Game"):
                games[title] = getattr(self.game_modules["g" + title], "Game")
        return games

    def meta_by_title(self, title):
        return self.game_meta.get(title)

    def game_by_title(self, title):
        return self.games.get(title)

In [ ]:
engine = Engine()

In [ ]:
game = engine.game_by_title("1830")

## Game Instantiation and basic tests

In [ ]:
g = game({"1": "Player 1", "2": "Player 2", "3": "Player 3", "4": "Player 4"})

In [ ]:
import inspect

test = False

skipped = []
methods = []
non_methods = []
for item_name in dir(g):
    if item_name.startswith("init") or item_name.startswith("_") or item_name.isupper():
        skipped.append(item_name)
        continue

    item = getattr(g, item_name)
    if callable(item):
        if inspect.signature(item).parameters:
            skipped.append(item)
        else:
            # print(item_name)
            methods.append(item_name)
            if test:
                item()
    else:
        # print(f"name: {item_name}, value: {item}")
        non_methods.append(item_name)

# methods, non_methods

## Actions Helper

In [ ]:
from rl18xx.game.engine.entities import Company
from rl18xx.game.engine.actions import (
    Bid,
    BuyCompany,
    BuyShares,
    BuyTrain,
    Dividend,
    LayTile,
    Par,
    Pass,
    PlaceToken,
    RunRoutes,
    SellShares,
)

from rl18xx.game.engine.abilities import Shares as SharesAbility
from rl18xx.game.engine.autorouter import AutoRouter
from rl18xx.game.engine.graph import Route
from rl18xx.game.engine.round import (
    BuyTrain as BuyTrainStep,
    Exchange as ExchangeStep,
    SpecialTrack as SpecialTrackStep,
    SpecialToken as SpecialTokenStep,
)

from collections import defaultdict

router = AutoRouter(g)


enabled = False


def print_summary():
    if enabled:
        for player in g.players:
            shares2 = {
                corporation: sum(share.percent for share in shares)
                for corporation, shares in player.shares_by_corporation.items()
            }
            print(player.name)
            print(f"    cash: {player.cash}")
            print(f"    shares: {shares2}")
            print(f"    companies: {player.companies}")

    print("")
    for corp in g.corporations:
        print(corp.name)
        print(f"    cash: {corp.cash}")
        print(f"    companies: {corp.companies}")
        print(f"    trains: {corp.trains}")
        print(f"    share price: {corp.share_price}")


def sort_actions(actions, instances=False):
    if instances:
        return sorted(
            actions,
            key=lambda x: x.__class__.__name__
            if x.__class__.__name__ != "Pass"
            else "zzzzzzzzzzzzzzzzzzzzzzzzzzzzzz",
        )

    return sorted(
        actions,
        key=lambda x: x.__name__
        if x.__name__ != "Pass"
        else "zzzzzzzzzzzzzzzzzzzzzzzzzzzzzz",
    )


def get_current_actions():
    if isinstance(g.current_entity, Company):
        if Pass in g.round.actions_for(g.current_entity):
            return [Pass]
        return []
    return sort_actions(g.round.actions_for(g.current_entity))


def get_company_actions():
    company_actions = {}
    if isinstance(g.current_entity, Company):
        company_actions[g.current_entity] = sort_actions(
            g.round.actions_for(g.current_entity)
        )
    elif hasattr(g.current_entity, "companies"):
        for company in g.current_entity.companies:
            company_actions[company] = sort_actions(g.round.actions_for(company))

    return company_actions


def get_corp_for_company(company):
    for ability in company.abilities:
        if isinstance(ability, SharesAbility):
            return ability.shares[0].corporation()
    return None


def get_bid_actions():
    companies = g.active_step().companies
    bids = []
    bids.append(
        [Bid(g.current_entity, g.active_step().min_bid(companies[0]), companies[0])]
    )
    for company in companies[1:]:
        min_bid = g.active_step().min_bid(company)
        max_bid = g.active_step().max_bid(g.current_entity, company)
        bid_values = list(
            range(min_bid - (min_bid % 5), (max_bid + 1) - ((max_bid + 1) % 5), 5)
        )
        bids.append(
            [Bid(g.current_entity, bid_value, company) for bid_value in bid_values]
        )
    return sorted([item for sublist in bids for item in sublist])


def get_par_actions():
    parable_corporations = []
    if hasattr(g.active_step(), "companies_pending_par"):
        par_values = g.share_prices
        return [
            Par(g.current_entity, get_corp_for_company(company), price)
            for company in g.active_step().companies_pending_par
            for price in par_values
        ]
    else:
        parable_corporations = sorted(
            [corp for corp in g.corporations if g.can_par(corp, g.current_entity)],
            key=lambda corporation: corporation.name,
        )
        par_values = g.share_prices
        buying_power = g.buying_power(g.current_entity)
        return sorted(
            [
                Par(g.current_entity, corp, price)
                for corp in parable_corporations
                for price in par_values
                if 2 * price.price <= buying_power
            ]
        )


def get_buy_shares_actions():
    buyable_shares = [
        item
        for sublist in g.active_step().buyable_shares(g.current_entity)
        for item in sublist
    ]
    unique_buyable_shares = sorted(
        list(set(buyable_shares)),
        key=lambda share: (share.corporation(), share.owner.__class__.__name__),
    )
    return sorted(
        [
            BuyShares(g.current_entity, share, share.price)
            for share in unique_buyable_shares
        ]
    )


def get_company_buy_shares_actions(company):
    exchange_step = [step for step in g.round.steps if isinstance(step, ExchangeStep)][
        0
    ]
    shares = exchange_step.exchangeable_shares(company)
    return sorted([BuyShares(company, share, share.price) for share in shares])


def get_sell_shares_actions():
    if isinstance(g.active_step(), BuyTrainStep):
        if (
            g.current_entity.is_corporation()
            and not g.current_entity.trains
            and g.current_entity.cash < g.active_step().needed_cash(g.current_entity)
        ):
            return [
                SellShares(bundle.owner, bundle)
                for bundle in g.active_step().sellable_shares(g.current_entity)
            ]
        else:
            return []
    else:
        return sorted(
            [
                SellShares(g.current_entity, bundle)
                for bundle in g.active_step().sellable_shares(g.current_entity)
            ]
        )


def get_place_token_actions():
    if hasattr(g.active_step(), "pending_token") and g.active_step().pending_token:
        hexes = g.active_step().pending_token.get("hexes")
        return sorted(
            [
                PlaceToken(g.current_entity, city, city.get_slot(g.current_entity))
                for hex in hexes
                for city in hex._tile.cities
            ]
        )

    return sorted(
        [
            PlaceToken(g.current_entity, city, city.get_slot(g.current_entity))
            for city in g.graph.connected_nodes(g.current_entity)
            if city.tokenable(g.current_entity)
        ]
    )


def get_company_place_token_actions(company):
    special_token_step = [
        step for step in g.round.steps if isinstance(step, SpecialTokenStep)
    ][0]
    ability = special_token_step.ability(company)

    actions = []
    for hex in ability.hexes:
        actions.extend(
            [
                PlaceToken(company, city, city.get_slot(g.current_entity))
                for city in g.hex_by_id(hex).tile.cities
            ]
        )
    return actions


def get_lay_tile_actions():
    moves = defaultdict(dict)
    for hex in g.graph.connected_hexes(g.current_entity):
        layable_tiles = []
        layable_tiles.extend(
            [
                tile
                for tile in g.active_step().upgradeable_tiles(g.current_entity, hex)
                if g.upgrade_cost(hex.tile, hex, g.current_entity, g.current_entity)
                <= g.buying_power(g.current_entity)
            ]
        )

        for tile in layable_tiles:
            rotations = g.active_step().legal_tile_rotations(
                g.current_entity, hex, tile
            )
            moves[hex][tile] = rotations
    return sorted(
        [
            LayTile(g.current_entity, tile, hex, rotation)
            for hex in moves
            for tile in moves[hex]
            for rotation in moves[hex][tile]
        ]
    )


def get_company_lay_tile_actions(company):
    special_track_step = [
        step for step in g.round.steps if isinstance(step, SpecialTrackStep)
    ][0]
    abilities = special_track_step.abilities(company)

    actions = []
    for ability in abilities:
        hexes = [g.hex_by_id(hex) for hex in ability.hexes]
        for hex in hexes:
            tiles = special_track_step.potential_tiles(company, hex)
            for tile in tiles:
                rotations = special_track_step.legal_tile_rotations(company, hex, tile)
                actions.extend(
                    [LayTile(company, tile, hex, rotation) for rotation in rotations]
                )

    return actions


def get_unique_trains(trains):
    unique_trains = {}
    for train in trains:
        key = (train.name, train.owner)
        if key not in unique_trains:
            unique_trains[key] = train
    return unique_trains.values()


def get_buy_train_actions():
    trains = get_unique_trains(g.depot.available(g.current_entity))
    actions = []
    for train in trains:
        min_price = train.min_price()
        if min_price > g.buying_power(g.current_entity):
            if g.active_step().must_buy_train(g.current_entity):
                if min_price > g.buying_power(g.current_entity) + g.buying_power(
                    g.current_entity.owner
                ):
                    continue
            else:
                continue
        max_price = train.price if train.from_depot() else g.current_entity.cash
        price_values = list(range(min_price, int(max_price) + 1))
        actions.append(
            [BuyTrain(g.current_entity, train, price) for price in price_values]
        )
    actions.append(get_exchange_train_actions())
    return sorted([item for sublist in actions for item in sublist])


def get_exchange_train_actions():
    discounted_trains = g.discountable_trains_for(g.current_entity)
    if not discounted_trains:
        return []

    unique_trains = get_unique_trains([discount[1] for discount in discounted_trains])
    unique_discounts = [
        discount for discount in discounted_trains if discount[1] in unique_trains
    ]
    return [
        BuyTrain(g.current_entity, discount[1], discount[3], exchange=discount[0])
        for discount in unique_discounts
    ]


def auto_route_action():
    best_routes = router.compute(g.current_entity, route_timeout=10000)
    return [
        RunRoutes(
            g.current_entity, [route for route in best_routes if route is not None]
        )
    ]


def get_dividend_actions():
    options = g.active_step().dividend_options(g.current_entity)
    actions = []
    for option in options.keys():
        actions.append(Dividend(g.current_entity, option))
    return sorted(actions)


def get_buy_company_actions():
    # Only buy from president
    companies = g.current_entity.owner.companies
    actions = []
    for company in companies:
        if not g.abilities(company, "no_buy"):
            actions.extend(
                [
                    BuyCompany(g.current_entity, company, price)
                    for price in range(company.min_price, company.max_price + 1)
                ]
            )
    return actions


def get_choices_for_action(action):
    if action == Pass:
        return [Pass(g.current_entity)]
    elif action == Bid:
        return get_bid_actions()
    elif action == Par:
        return get_par_actions()
    elif action == BuyShares:
        return get_buy_shares_actions()
    elif action == SellShares:
        return get_sell_shares_actions()
    elif action == PlaceToken:
        return get_place_token_actions()
    elif action == LayTile:
        return get_lay_tile_actions()
    elif action == BuyTrain:
        return get_buy_train_actions()
    elif action == RunRoutes:
        return auto_route_action()
    elif action == Dividend:
        return get_dividend_actions()
    elif action == BuyCompany:
        return get_buy_company_actions()
    else:
        return []


def get_company_choices():
    company_actions = get_company_actions()
    if not company_actions.values():
        return []

    choices = []
    for company, actions in company_actions.items():
        for action in actions:
            if action == BuyShares:
                choices.extend(get_company_buy_shares_actions(company))
            elif action == LayTile:
                choices.extend(get_company_lay_tile_actions(company))
            elif action == PlaceToken:
                choices.extend(get_company_place_token_actions(company))
    return choices


def get_all_choices():
    choices = [
        choices
        for action in get_current_actions()
        for choices in get_choices_for_action(action)
    ]
    choices.extend(get_company_choices())
    return sort_actions(choices, instances=True)


def get_all_choices_with_index():
    return {index: value for index, value in enumerate(get_all_choices())}

In [ ]:
# get_all_choices()
# get_company_choices()

## Waterfall auction

In [ ]:
g.process_action(get_all_choices()[-1])  # pass
g.process_action(get_all_choices()[1])  # bid 45 on CS
g.process_action(get_all_choices()[1])  # bid 50 on CS
g.process_action(get_all_choices()[-76])  # bid 225 on BO
g.process_action(get_all_choices()[0])  # buy SV
g.process_action(get_all_choices()[-1])  # pass
g.process_action(get_all_choices()[0])  # buy DH
g.process_action(get_all_choices()[0])  # buy MH
g.process_action(get_all_choices()[0])  # buy CA
g.process_action(get_all_choices()[0])  # Par B&O at 100

<rl18xx.game.engine.game.title.g1830.Game>

In [ ]:
print_summary()


PRR
    cash: 0
    companies: []
    trains: []
    share price: None
NYC
    cash: 0
    companies: []
    trains: []
    share price: None
CPR
    cash: 0
    companies: []
    trains: []
    share price: None
B&O
    cash: 0
    companies: []
    trains: []
    share price: Share Price - coordinates: (0, 6), price: 100, types: ['par'], corporations: [<Corporation: B&O>]
C&O
    cash: 0
    companies: []
    trains: []
    share price: None
ERIE
    cash: 0
    companies: []
    trains: []
    share price: None
NYNH
    cash: 0
    companies: []
    trains: []
    share price: None
B&M
    cash: 0
    companies: []
    trains: []
    share price: None


## Stock Round 1

In [ ]:
g.process_action(get_all_choices()[-2])  # Par PRR
g.process_action(get_all_choices()[-1])  # Pass
g.process_action(get_all_choices()[-8])  # Par NYC
g.process_action(get_all_choices()[1])  # Buy PRR
g.process_action(get_all_choices()[1])  # Buy PRR
g.process_action(get_all_choices()[14])  # Par C&O
g.process_action(get_all_choices()[2])  # Buy NYC
g.process_action(get_all_choices()[1])  # Buy PRR
g.process_action(get_all_choices()[1])  # Buy PRR
g.process_action(get_all_choices()[3])  # Buy C&O
g.process_action(get_all_choices()[2])  # Buy NYC
g.process_action(get_all_choices()[0])  # Buy PRR
g.process_action(get_all_choices()[1])  # Buy PRR
g.process_action(get_all_choices()[3])  # Buy C&O
g.process_action(get_all_choices()[2])  # Buy NYC
g.process_action(get_all_choices()[1])  # Buy PRR
g.process_action(get_all_choices()[2])  # Buy C&O
g.process_action(get_all_choices()[1])  # Buy NYC
g.process_action(get_all_choices()[1])  # Buy NYC
g.process_action(get_all_choices()[2])  # Buy C&O
g.process_action(get_all_choices()[1])  # Buy NYC
g.process_action(get_all_choices()[1])  # Buy NYC
# Player 1: 60% PRR, 20% NYC
# Player 2: 60% C&O, 10% NYC
# Player 3: 60% NYC
# Player 4: 40% PRR, 20% B&O

<rl18xx.game.engine.game.title.g1830.Game>

In [ ]:
print_summary()


PRR
    cash: 670.0
    companies: []
    trains: []
    share price: Share Price - coordinates: (4, 6), price: 71, types: ['par'], corporations: [<Corporation: PRR>]
NYC
    cash: 670.0
    companies: []
    trains: []
    share price: Share Price - coordinates: (5, 6), price: 67, types: ['par'], corporations: [<Corporation: NYC>, <Corporation: C&O>]
CPR
    cash: 0
    companies: []
    trains: []
    share price: None
B&O
    cash: 0
    companies: []
    trains: []
    share price: Share Price - coordinates: (0, 6), price: 100, types: ['par'], corporations: [<Corporation: B&O>]
C&O
    cash: 670.0
    companies: []
    trains: []
    share price: Share Price - coordinates: (5, 6), price: 67, types: ['par'], corporations: [<Corporation: NYC>, <Corporation: C&O>]
ERIE
    cash: 0
    companies: []
    trains: []
    share price: None
NYNH
    cash: 0
    companies: []
    trains: []
    share price: None
B&M
    cash: 0
    companies: []
    trains: []
    share price: None


## Operating Round 1

In [ ]:
# PRR
g.process_action(get_all_choices()[0])  # lays tile #57 with rotation 1 on H10
g.process_action(get_all_choices()[-1])  # passes place token
g.process_action(get_all_choices()[0])  # buys a 2 train
g.process_action(get_all_choices()[0])  # buys a 2 train
g.process_action(get_all_choices()[-1])  # passes trains

# NYC
g.process_action(get_all_choices()[0])  # lays tile #57 with rotation 0 on E19
g.process_action(get_all_choices()[0])  # buys a 2 train
g.process_action(get_all_choices()[-1])  # passes trains

# C&O
g.process_action(get_all_choices()[-5])
g.process_action(get_all_choices()[0])
g.process_action(get_all_choices()[1])

> /home/revys/workspace/RL_18xx/rl18xx/game/engine/graph.py(16)copy_dict()
     14 def copy_dict(dict):
     15     set_trace()
---> 16     return {key: value for key, value in dict.items()}
     17 
     18 # %% ../../../nbs/game/engine/05_graph.ipynb 8



ipdb>  a


dict = {}


ipdb>  c


> /home/revys/workspace/RL_18xx/rl18xx/game/engine/graph.py(16)copy_dict()
     14 def copy_dict(dict):
     15     set_trace()
---> 16     return {key: value for key, value in dict.items()}
     17 
     18 # %% ../../../nbs/game/engine/05_graph.ipynb 8



...

> /home/revys/workspace/RL_18xx/rl18xx/game/engine/graph.py(16)copy_dict()
     14 def copy_dict(dict):
     15     set_trace()
---> 16     return {key: value for key, value in dict.items()}
     17 
     18 # %% ../../../nbs/game/engine/05_graph.ipynb 8



ipdb>  a


dict = {}


ipdb>  c


<rl18xx.game.engine.game.title.g1830.Game>

In [ ]:
print_summary()


PRR
    cash: 510.0
    companies: []
    trains: [<Train: 2-0, owner: <Corporation: PRR>>, <Train: 2-1, owner: <Corporation: PRR>>]
    share price: Share Price - coordinates: (4, 5), price: 67, types: [], corporations: [<Corporation: PRR>]
NYC
    cash: 590.0
    companies: []
    trains: [<Train: 2-2, owner: <Corporation: NYC>>]
    share price: Share Price - coordinates: (5, 5), price: 65, types: [], corporations: [<Corporation: NYC>, <Corporation: C&O>]
CPR
    cash: 0
    companies: []
    trains: []
    share price: None
B&O
    cash: 0
    companies: []
    trains: []
    share price: Share Price - coordinates: (0, 6), price: 100, types: ['par'], corporations: [<Corporation: B&O>]
C&O
    cash: 590.0
    companies: []
    trains: [<Train: 2-3, owner: <Corporation: C&O>>]
    share price: Share Price - coordinates: (5, 5), price: 65, types: [], corporations: [<Corporation: NYC>, <Corporation: C&O>]
ERIE
    cash: 0
    companies: []
    trains: []
    share price: None
NYNH
   

## Stock Round 2

In [ ]:
g.process_action(get_all_choices()[-2])  # sell nyc
g.process_action(get_all_choices()[-3])  # par nynh

g.process_action(get_all_choices()[0])  # buy C&O
g.process_action(get_all_choices()[-1])  # pass sell

g.process_action(get_all_choices()[0])  # Buy NYC
g.process_action(get_all_choices()[-1])  # pass sell

g.process_action(get_all_choices()[0])  # Buy NYNH
g.process_action(get_all_choices()[-1])  # pass sell

g.process_action(get_all_choices()[1])  # Buy NYNH
g.process_action(get_all_choices()[-1])  # pass sell

g.process_action(get_all_choices()[-1])  # pass

g.process_action(get_all_choices()[-1])  # pass

g.process_action(get_all_choices()[-1])  # pass

g.process_action(get_all_choices()[1])  # Buy NYNH
g.process_action(get_all_choices()[-1])  # pass sell

g.process_action(get_all_choices()[-1])  # pass

g.process_action(get_all_choices()[-1])  # pass

g.process_action(get_all_choices()[-1])  # pass

g.process_action(get_all_choices()[1])  # Buy NYNH
g.process_action(get_all_choices()[-1])  # pass sell

g.process_action(get_all_choices()[-1])  # pass

g.process_action(get_all_choices()[-1])  # pass

g.process_action(get_all_choices()[-1])  # pass

g.process_action(get_all_choices()[-1])  # pass
# Player 1: 60% PRR, 30% NYC
# Player 2: 60% C&O, 10% NYC, 10% NYNH
# Player 3: 50% NYNH, 10% NYC
# Player 4: 40% PRR, 20% B&O, 10% C&O

<rl18xx.game.engine.game.title.g1830.Game>

In [ ]:
print_summary()


PRR
    cash: 510.0
    companies: []
    trains: [<Train: 2-0, owner: <Corporation: PRR>>, <Train: 2-1, owner: <Corporation: PRR>>]
    share price: Share Price - coordinates: (3, 5), price: 71, types: [], corporations: [<Corporation: PRR>]
NYC
    cash: 590.0
    companies: []
    trains: [<Train: 2-2, owner: <Corporation: NYC>>]
    share price: Share Price - coordinates: (10, 5), price: 30, types: ['multiple_buy'], corporations: [<Corporation: NYC>]
CPR
    cash: 0
    companies: []
    trains: []
    share price: None
B&O
    cash: 0
    companies: []
    trains: []
    share price: Share Price - coordinates: (0, 6), price: 100, types: ['par'], corporations: [<Corporation: B&O>]
C&O
    cash: 590.0
    companies: []
    trains: [<Train: 2-3, owner: <Corporation: C&O>>]
    share price: Share Price - coordinates: (5, 5), price: 65, types: [], corporations: [<Corporation: C&O>]
ERIE
    cash: 0
    companies: []
    trains: []
    share price: None
NYNH
    cash: 710.0
    companie

## Operating Round 2

In [ ]:
# NYNH
g.process_action(get_all_choices()[0])  # lay #1 with rotation 0 on F20
g.process_action(get_all_choices()[0])  # buy 2 train
g.process_action(get_all_choices()[-1])  # pass trains

# PRR
g.process_action(get_all_choices()[4])  # lay tile #9 with rotation 1 on H8
g.process_action(get_all_choices()[-1])  # pass token
g.process_action(get_all_choices()[0])  # auto trains & run
g.process_action(get_all_choices()[0])  # pay out
g.process_action(get_all_choices()[-1])  # pass trains

# C&O
g.process_action(get_all_choices()[-3])  # lay tile #8 with rotation 2 on G3
g.process_action(get_all_choices()[0])  # auto trains & run
g.process_action(get_all_choices()[1])  # withhold
g.process_action(get_all_choices()[0])  # buy a 2 train
g.process_action(get_all_choices()[0])  # buy a 3 train
g.process_action(get_all_choices()[-1])  # pass trains
g.process_action(get_all_choices()[-2])  # buy DH from Player 2 for $140
g.process_action(get_all_choices()[0])  # pass buy companies

# NYC
g.process_action(get_all_choices()[-10])  # lay tile #8 with rotation 3 on F18
g.process_action(get_all_choices()[31])  # buy 3 train
g.process_action(get_all_choices()[-1])  # pass trains
g.process_action(get_all_choices()[-2])  # buys Schuylkill Valley from Player 1 for $40
g.process_action(get_all_choices()[-1])  # pass buy companies

> /home/revys/workspace/RL_18xx/rl18xx/game/engine/graph.py(16)copy_dict()
     14 def copy_dict(dict):
     15     set_trace()
---> 16     return {key: value for key, value in dict.items()}
     17 
     18 # %% ../../../nbs/game/engine/05_graph.ipynb 8



In [ ]:
print_summary()

## Stock Round 3

In [ ]:
g.process_action(get_all_choices()[2])  # Buy NYC
g.process_action(get_all_choices()[-1])  # Pass Sell

g.process_action(get_all_choices()[3])  # Buy NYC
g.process_action(get_all_choices()[-1])  # Pass Sell

g.process_action(get_all_choices()[3])  # Buy NYC
g.process_action(get_all_choices()[-1])  # Pass Sell

g.process_action(get_all_choices()[3])  # Exchange for NYC
g.process_action(get_all_choices()[1])  # Buy last NYC
g.process_action(get_all_choices()[-1])  # Pass sell

g.process_action(get_all_choices()[-1])  # pass

g.process_action(get_all_choices()[-1])  # pass

g.process_action(get_all_choices()[1])  # Buy NYNH
g.process_action(get_all_choices()[-1])  # pass

g.process_action(get_all_choices()[-1])  # pass

g.process_action(get_all_choices()[-1])  # pass

g.process_action(get_all_choices()[-1])  # pass

g.process_action(get_all_choices()[-1])  # pass

In [ ]:
print_summary()

## Operating Round 3

In [ ]:
# PRR
g.process_action(get_all_choices()[-11])  # lay tile 8 with rotation 2 on H6
g.process_action(get_all_choices()[0])  # Token Pittsburgh
g.process_action(get_all_choices()[0])  # Run 2 train for 30
g.process_action(get_all_choices()[0])  # pay out
g.process_action(get_all_choices()[-1])  # pass trains
g.process_action(get_all_choices()[-1])  # pass companies

# NYNH
g.process_action(get_all_choices()[-5])  # Buy C&S
g.process_action(get_all_choices()[0])  # lay tile 54 with rotation 0 on G19
g.process_action(get_all_choices()[-1])  # run routes
g.process_action(get_all_choices()[0])  # pay out
g.process_action(get_all_choices()[0])  # buy 3 train
g.process_action(get_all_choices()[-1])  # pass trains
g.process_action(get_all_choices()[-4])  # lay C&S tile
g.process_action(get_all_choices()[-1])  # pass companies

# C&O
g.process_action(get_all_choices()[-6])  # Lay tile 57 with rotation 1 on F16
g.process_action(get_all_choices()[0])  # Token teleport
g.process_action(get_all_choices()[0])  # run routes
g.process_action(get_all_choices()[0])  # pay out
g.process_action(get_all_choices()[-1])  # pass companies

# NYC
g.process_action(get_all_choices()[8])  # lay tile 19 with rotation 1 on F18
g.process_action(get_all_choices()[0])  # run trains
g.process_action(get_all_choices()[0])  # pay out
g.process_action(get_all_choices()[-1])  # pass trains
g.process_action(get_all_choices()[-1])  # pass companies

# PRR
g.process_action(get_all_choices()[-3])  # lay tile 25 with rotation 1 on g5
g.process_action(get_all_choices()[0])  # run routes
g.process_action(get_all_choices()[0])  # pay out
g.process_action(get_all_choices()[-1])  # pass trains
g.process_action(get_all_choices()[-1])  # pass companies

# NYNH
g.process_action(get_all_choices()[0])  # lay tile 14 with rotation 0 on E19
g.process_action(get_all_choices()[0])  # place a token on E19
g.process_action(get_all_choices()[0])  # run trains
g.process_action(get_all_choices()[0])  # pay out
g.process_action(get_all_choices()[-1])  # pass trains
g.process_action(get_all_choices()[-1])  # pass companies

# C&O
g.process_action(get_all_choices()[5])  # lay tile 15 with rotation 4 on F16
g.process_action(get_all_choices()[0])  # run trains
g.process_action(get_all_choices()[0])  # pay out
g.process_action(get_all_choices()[-1])  # pass companies

# NYC
g.process_action(get_all_choices()[-7])  # lay tile #7 with rotation 0 on E21
g.process_action(get_all_choices()[0])  # place token on F16
g.process_action(get_all_choices()[0])  # run trains
g.process_action(get_all_choices()[0])  # pay out
g.process_action(get_all_choices()[-1])  # pass trains
g.process_action(get_all_choices()[-1])  # pass companies

In [ ]:
print_summary()

## Stock Round 4

In [ ]:
g.process_action(get_all_choices()[0])  # buy B&O
g.process_action(get_all_choices()[-1])  # pass sell

g.process_action(get_all_choices()[0])  # buy B&O
g.process_action(get_all_choices()[-1])  # pass sell

g.process_action(get_all_choices()[2])  # buy C&O
g.process_action(get_all_choices()[-1])  # pass sell

g.process_action(get_all_choices()[1])  # buy NYNH
g.process_action(get_all_choices()[-1])  # pass sell

g.process_action(get_all_choices()[0])  # buy B&O
g.process_action(get_all_choices()[-1])  # pass sell

g.process_action(get_all_choices()[0])  # buy B&O
g.process_action(get_all_choices()[-1])  # pass sell

g.process_action(get_all_choices()[2])  # buy C&O
g.process_action(get_all_choices()[-1])  # pass sell

g.process_action(get_all_choices()[1])  # buy NYNH
g.process_action(get_all_choices()[-1])  # pass sell

g.process_action(get_all_choices()[0])  # buy B&O
g.process_action(get_all_choices()[-1])  # pass sell

g.process_action(get_all_choices()[-1])  # pass

g.process_action(get_all_choices()[1])  # buy C&O
g.process_action(get_all_choices()[-1])  # pass sell

g.process_action(get_all_choices()[1])  # buy NYNH
g.process_action(get_all_choices()[-1])  # pass sell

g.process_action(get_all_choices()[-1])  # pass
g.process_action(get_all_choices()[-1])  # pass
g.process_action(get_all_choices()[-1])  # pass
g.process_action(get_all_choices()[-1])  # pass

In [ ]:
print_summary()

## Operating Round 4

In [ ]:
# PRR
g.process_action(get_all_choices()[9])  # lay tile #8 with rotation 5 on H14
g.process_action(get_all_choices()[0])  # Run trains
g.process_action(get_all_choices()[0])  # pay out
g.process_action(get_all_choices()[0])  # buy 3 train
g.process_action(get_all_choices()[-1])  # pass trains
g.process_action(get_all_choices()[-1])  # pass companies

# B&O
g.process_action(get_all_choices()[-12])  # buy C&A for max
g.process_action(get_all_choices()[5])  # lay tile #8 with rotation 1 on I17
g.process_action(get_all_choices()[0])  # buy 3 train
g.process_action(get_all_choices()[0])  # buy 4 train

# NYNH
g.process_action(get_all_choices()[-3])  # lay tile #57 with rotation 1 on F22
g.process_action(get_all_choices()[0])  # run trains
g.process_action(get_all_choices()[0])  # pay out

# C&O
g.process_action(get_all_choices()[-20])  # lay tile #56 with rotation 1 on G17
g.process_action(get_all_choices()[0])  # place token on G19
g.process_action(get_all_choices()[0])  # run trains
g.process_action(get_all_choices()[0])  # pay out

# NYC
g.process_action(get_all_choices()[13])  # lay tile #26 with rotation 1 on E21
g.process_action(get_all_choices()[0])  # run trains
g.process_action(get_all_choices()[0])  # pay out
g.process_action(get_all_choices()[109])  # buy a 3 train for $110 from PRR

# PRR
g.process_action(get_all_choices()[12])  # lay tile #53 with rotation 0 on I15
g.process_action(get_all_choices()[0])  # buy 4 train
g.process_action(get_all_choices()[-1])  # pass trains

# NYNH
g.process_action(get_all_choices()[12])  # tile #53 with rotation 1 on E23
g.process_action(get_all_choices()[0])  # runs a 3 train for $140: G19-E19-E23
g.process_action(get_all_choices()[1])  # withholds $140
g.process_action(get_all_choices()[0])  # buys a 4 train for $300 from The Depot

# B&O
g.process_action(get_all_choices()[-2])  # lay tile #59 with rotation 0 on H18
g.process_action(get_all_choices()[-1])  # skip token
g.process_action(get_all_choices()[0])  # run trains
g.process_action(get_all_choices()[0])  # pay out


# C&O
g.process_action(get_all_choices()[8])  # lay tile #69 with rotation 0 on G7
g.process_action(get_all_choices()[0])  # run trains
g.process_action(get_all_choices()[0])  # pay out

# NYC
g.process_action(get_all_choices()[-3])  # lay tile 9 with rotation 1 on F14
g.process_action(get_all_choices()[-1])  # skip token
g.process_action(get_all_choices()[0])  # run trains
g.process_action(get_all_choices()[0])  # pay out
g.process_action(get_all_choices()[-1])  # pass trains

g.log

In [ ]:
print_summary()

## Stock Round 5

In [ ]:
g.process_action(
    get_all_choices()[0]
)  # [18:00] Player 3 buys a 10% share of B&O from the IPO for $100
g.process_action(get_all_choices()[-1])  # [18:00] Player 3 declines to sell shares
g.process_action(
    get_all_choices()[0]
)  # [18:00] Player 4 buys a 10% share of B&O from the IPO for $100
g.process_action(get_all_choices()[-1])  # [18:00] Player 4 declines to sell shares
g.process_action(
    get_all_choices()[0]
)  # [18:00] Player 1 buys a 10% share of B&O from the IPO for $100
g.process_action(get_all_choices()[-1])  # [18:00] Player 1 declines to sell shares
g.process_action(get_all_choices()[12])  # [18:01] Player 2 pars ERIE at $100
# [18:01] Player 2 buys a 20% share of ERIE from the IPO for $200
# [18:01] Player 2 becomes the president of ERIE
g.process_action(get_all_choices()[-1])  # [18:01] Player 2 declines to sell shares
g.process_action(
    get_all_choices()[0]
)  # [18:01] Player 3 buys a 10% share of ERIE from the IPO for $100
g.process_action(get_all_choices()[-1])  # [18:04] Player 3 declines to sell shares
g.process_action(
    get_all_choices()[0]
)  # [18:04] Player 4 buys a 10% share of ERIE from the IPO for $100
g.process_action(get_all_choices()[-1])  # [18:04] Player 4 declines to sell shares
g.process_action(
    get_all_choices()[0]
)  # [18:04] Player 1 buys a 10% share of ERIE from the IPO for $100
g.process_action(get_all_choices()[-1])  # [18:04] Player 1 declines to sell shares
g.process_action(
    get_all_choices()[0]
)  # [18:05] Player 2 sells a 10% share of NYC and receives $68
g.process_action(
    get_all_choices()[0]
)  # [18:05] Player 2 buys a 10% share of ERIE from the IPO for $100
# [18:05] ERIE floats
# [18:05] ERIE receives $1000
g.process_action(
    get_all_choices()[0]
)  # [18:05] Player 2 sells a 10% share of NYC and receives $68
g.process_action(get_all_choices()[-1])  # [18:05] Player 2 passes
g.process_action(get_all_choices()[-1])  # [18:05] Player 3 passes
g.process_action(
    get_all_choices()[0]
)  # [18:05] Player 4 buys a 10% share of ERIE from the IPO for $100
g.process_action(get_all_choices()[-1])  # [18:05] Player 4 declines to sell shares
g.process_action(
    get_all_choices()[0]
)  # [18:05] Player 1 buys a 10% share of NYC from the market for $68
g.process_action(get_all_choices()[-1])  # [18:05] Player 1 declines to sell shares
g.process_action(
    get_all_choices()[0]
)  # [18:06] Player 2 buys a 10% share of ERIE from the IPO for $100
g.process_action(get_all_choices()[-1])  # [18:06] Player 2 declines to sell shares
g.process_action(get_all_choices()[-1])  # [18:06] Player 3 passes
g.process_action(
    get_all_choices()[0]
)  # [18:06] Player 4 buys a 10% share of ERIE from the IPO for $100
g.process_action(get_all_choices()[-1])  # [18:06] Player 4 declines to sell shares
g.process_action(get_all_choices()[-1])  # [18:06] Player 1 passes
g.process_action(get_all_choices()[-1])  # [18:06] Player 2 passes
g.process_action(get_all_choices()[-1])  # [18:06] Player 3 passes
g.process_action(
    get_all_choices()[0]
)  # [18:06] Player 4 buys a 10% share of ERIE from the IPO for $100
g.process_action(get_all_choices()[-1])  # [18:06] Player 4 declines to sell shares
g.process_action(get_all_choices()[-1])  # [18:06] Player 1 passes
g.process_action(get_all_choices()[-1])  # [18:06] Player 2 passes
g.process_action(get_all_choices()[-1])  # [18:06] Player 3 passes
g.process_action(
    get_all_choices()[0]
)  # [18:06] Player 4 buys a 10% share of NYC from the market for $68
g.process_action(get_all_choices()[-1])  # [18:06] Player 4 declines to sell shares
g.process_action(get_all_choices()[-1])  # pass
g.process_action(get_all_choices()[-1])  # pass
g.process_action(get_all_choices()[-1])  # pass
g.process_action(get_all_choices()[-1])  # pass

In [ ]:
print_summary()

## Operating Round 5

In [ ]:
# [18:06] B&O collects $25 from Camden & Amboy
# [18:06] C&O collects $15 from Delaware & Hudson
# [18:06] NYNH collects $10 from Champlain & St.Lawrence
# [18:06] NYC collects $5 from Schuylkill Valley


# [18:06] Player 1 operates PRR
g.process_action(
    get_all_choices()[4]
)  # [18:12] PRR lays tile #15 with rotation 1 on H10 (Pittsburgh)
# [18:12] PRR skips place a token
g.process_action(
    get_all_choices()[0]
)  # [18:12] PRR runs a 4 train for $130: F2-H10-H12-I15
g.process_action(
    get_all_choices()[0]
)  # [18:12] PRR pays out $130 = $13 per share ($78 to Player 1, $52 to Player 4)
# [18:12] PRR's share price moves right from $126 to $142
g.process_action(get_all_choices()[-1])  # [18:12] PRR passes buy trains
# [18:12] PRR skips buy companies


# [18:12] Player 4 operates B&O
g.process_action(
    get_all_choices()[0]
)  # [18:13] B&O spends $80 and lays tile #57 with rotation 0 on J14 (Washington)
g.process_action(
    get_all_choices()[1]
)  # [18:13] B&O places a token on H10 (Pittsburgh) for $40
g.process_action(
    get_all_choices()[0]
)  # [18:13] B&O runs a 4 train for $140: K13-J14-I15-H18
# [18:13] B&O runs a 3 train for $120: I15-H10-F2
g.process_action(
    get_all_choices()[0]
)  # [18:13] B&O pays out $260 = $26 per share ($130 to Player 4, $104 to Player 3, $26 to Player 1)
# [18:13] B&O's share price moves right from $100 to $112
# [18:13] B&O passes buy trains
# [18:13] B&O skips buy companies


# [18:13] Player 2 operates ERIE
# [18:13] ERIE places a token on E11
g.process_action(
    get_all_choices()[2]
)  # [18:13] ERIE lays tile #59 with rotation 3 on E11 (Dunkirk & Buffalo)
# [18:13] ERIE must choose city for token
g.process_action(
    get_all_choices()[1]
)  # [18:13] ERIE places a token on E11 (Dunkirk & Buffalo)
# [18:13] ERIE skips place a token
# [18:13] ERIE skips run routes
# [18:13] ERIE does not run
# [18:13] ERIE's share price moves left from $100 to $90
g.process_action(
    get_all_choices()[0]
)  # [18:13] ERIE buys a 4 train for $300 from The Depot
g.process_action(
    get_all_choices()[0]
)  # [18:13] ERIE buys a 5 train for $450 from The Depot
# [18:13] -- Phase 5 (Operating Rounds: 3 | Train Limit: 2 | Available Tiles: Yellow, Green, Brown) --
# [18:13] -- Event: Private companies close --
# [18:13] ERIE skips buy companies


# [18:13] Player 2 operates C&O
g.process_action(
    get_all_choices()[-4]
)  # [18:16] C&O lays tile #66 with rotation 0 on H18 (Philadelphia & Trenton)
# [18:16] C&O skips place a token
g.process_action(
    get_all_choices()[0]
)  # [18:16] C&O runs a 3 train for $160: G19-H18-I15
g.process_action(get_all_choices()[1])  # [18:16] C&O withholds $160
# [18:16] C&O's share price moves left from $90 to $82
g.process_action(
    get_all_choices()[474]
)  # [18:17] C&O buys a 5 train for $200 from ERIE
# [18:17] C&O skips buy companies


# [18:17] Player 3 operates NYNH
g.process_action(
    get_all_choices()[0]
)  # [18:19] NYNH lays tile #63 with rotation 0 on E19 (Albany)
# [18:19] NYNH skips place a token
g.process_action(
    get_all_choices()[0]
)  # [18:19] NYNH runs a 4 train for $100: G19-F20-F22-F24
# [18:19] NYNH runs a 3 train for $150: G19-E19-E23
g.process_action(
    get_all_choices()[0]
)  # [18:21] NYNH pays out $250 = $25 per share ($125 to Player 2, $125 to Player 3)
# [18:21] NYNH's share price moves right from $90 to $100
# [18:21] NYNH skips buy trains
# [18:21] NYNH skips buy companies


# [18:21] Player 1 operates NYC
g.process_action(
    get_all_choices()[8]
)  # [18:21] NYC lays tile #62 with rotation 0 on G19 (New York & Newark)
g.process_action(
    get_all_choices()[3]
)  # [18:21] NYC places a token on H18 (Philadelphia & Trenton) for $100
g.process_action(
    get_all_choices()[0]
)  # [18:21] NYC runs a 3 train for $170: G19-E19-E23
# [18:21] NYC runs a 3 train for $180: G19-H18-I15
g.process_action(get_all_choices()[1])  # [18:21] NYC withholds $350
# [18:21] NYC's share price moves left from $69 to $67
# [18:21] NYC skips buy trains
# [18:21] NYC skips buy companies


# [18:21] -- Operating Round 5.2 (of 2) --
# [18:21] Player 1 operates PRR
g.process_action(
    get_all_choices()[-3]
)  # [18:21] PRR lays tile #9 with rotation 0 on G11
# [18:21] PRR skips place a token
g.process_action(
    get_all_choices()[0]
)  # [18:21] PRR runs a 4 train for $160: F2-H10-H12-I15
g.process_action(get_all_choices()[1])  # [18:21] PRR withholds $160
# [18:21] PRR's share price moves left from $142 to $126
g.process_action(get_all_choices()[-1])  # [18:21] PRR passes buy trains
# [18:21] PRR skips buy companies


# [18:21] Player 4 operates B&O
g.process_action(
    get_all_choices()[6]
)  # [18:22] B&O lays tile #61 with rotation 0 on I15 (Baltimore)
# [18:22] B&O skips place a token
g.process_action(
    get_all_choices()[0]
)  # [18:22] B&O runs a 4 train for $170: K13-J14-I15-H18
# [18:22] B&O runs a 3 train for $160: I15-H10-F2
g.process_action(get_all_choices()[1])  # [18:22] B&O withholds $330
# [18:22] B&O's share price moves left from $112 to $100
# [18:22] B&O skips buy trains
# [18:22] B&O skips buy companies


# [18:22] Player 3 operates NYNH
g.process_action(
    get_all_choices()[5]
)  # [18:22] NYNH lays tile #8 with rotation 5 on D18
# [18:22] NYNH skips place a token
g.process_action(
    get_all_choices()[0]
)  # [18:22] NYNH runs a 4 train for $120: G19-F20-F22-F24
# [18:22] NYNH runs a 3 train for $170: G19-E19-E23
g.process_action(
    get_all_choices()[0]
)  # [18:22] NYNH pays out $290 = $29 per share ($145 to Player 2, $145 to Player 3)
# [18:22] NYNH's share price moves right from $100 to $111
# [18:22] NYNH skips buy trains
# [18:22] NYNH skips buy companies


# [18:22] Player 2 operates ERIE
g.process_action(
    get_all_choices()[5]
)  # [18:22] ERIE lays tile #8 with rotation 2 on F12
# [18:22] ERIE skips place a token
g.process_action(get_all_choices()[0])  # [18:22] ERIE runs a 4 train for $70: E11-F16
g.process_action(
    get_all_choices()[0]
)  # [18:23] ERIE pays out $70 = $7 per share ($28 to Player 2, $28 to Player 4, $7 to Player 1, $7 to Player 3)
# [18:23] ERIE's share price moves right from $90 to $100
g.process_action(
    get_all_choices()[0]
)  # [18:23] ERIE buys a 5 train for $450 from The Depot
# [18:23] ERIE skips buy companies


# [18:23] Player 2 operates C&O
g.process_action(
    get_all_choices()[-3]
)  # [18:23] C&O lays tile #68 with rotation 2 on E11 (Dunkirk & Buffalo)
# [18:23] C&O skips place a token
g.process_action(
    get_all_choices()[0]
)  # [18:23] C&O runs a 5 train for $220: E11-F16-G17-G19-H18
# [18:23] C&O runs a 3 train for $110: F2-F6-G7
g.process_action(
    get_all_choices()[0]
)  # [18:23] C&O pays out $330 = $33 per share ($198 to Player 2, $99 to Player 1, $33 to Player 4)
# [18:23] C&O's share price moves right from $82 to $90
# [18:23] C&O skips buy trains
# [18:23] C&O skips buy companies


# [18:23] Player 1 operates NYC
g.process_action(
    get_all_choices()[41]
)  # [18:23] NYC lays tile #9 with rotation 1 on D16
g.process_action(get_all_choices()[-1])  # [18:23] NYC passes place a token
g.process_action(
    get_all_choices()[0]
)  # [18:23] NYC runs a 3 train for $170: G19-E19-E23
# [18:23] NYC runs a 3 train for $190: G19-H18-I15
g.process_action(get_all_choices()[1])  # [18:23] NYC withholds $360
# [18:23] NYC's share price moves left from $67 to $63
# [18:23] NYC skips buy trains
# [18:23] NYC skips buy companies

In [ ]:
print_summary()

## Stock Round 6

In [ ]:
g.process_action(get_all_choices()[0])  # [19:11] Player 1 pars B&M at $100
# [19:11] Player 1 buys a 20% share of B&M from the IPO for $200
# [19:11] Player 1 becomes the president of B&M
g.process_action(get_all_choices()[-1])  # [19:11] Player 1 declines to sell shares
g.process_action(get_all_choices()[1])  # [19:12] Player 2 pars CPR at $100
# [19:12] Player 2 buys a 20% share of CPR from the IPO for $200
# [19:12] Player 2 becomes the president of CPR
g.process_action(
    get_all_choices()[9]
)  # [19:13] Player 2 sells a 10% share of NYNH and receives $111
# [19:13] NYNH's share price moves down from $111 to $100
g.process_action(get_all_choices()[-1])
g.process_action(
    get_all_choices()[2]
)  # [19:14] Player 3 buys a 10% share of CPR from the IPO for $100
g.process_action(get_all_choices()[-1])  # [19:14] Player 3 declines to sell shares
g.process_action(
    get_all_choices()[2]
)  # [19:14] Player 4 buys a 10% share of CPR from the IPO for $100
g.process_action(get_all_choices()[-1])  # [19:14] Player 4 declines to sell shares
g.process_action(
    get_all_choices()[2]
)  # [19:15] Player 1 sells 3 shares of PRR and receives $378
# [19:15] Player 4 becomes the president of PRR
# [19:15] PRR's share price moves down from $126 to $90
g.process_action(
    get_all_choices()[-2]
)  # [19:15] Player 1 sells a 10% share of ERIE and receives $100
# [19:15] ERIE's share price moves down from $100 to $90
g.process_action(
    get_all_choices()[1]
)  # [19:15] Player 1 buys a 10% share of B&M from the IPO for $100
g.process_action(get_all_choices()[-1])
g.process_action(
    get_all_choices()[3]
)  # [19:15] Player 2 buys a 10% share of ERIE from the market for $90
g.process_action(get_all_choices()[-1])  # [19:15] Player 2 declines to sell shares
g.process_action(
    get_all_choices()[0]
)  # [19:15] Player 3 buys a 10% share of NYNH from the market for $100
g.process_action(get_all_choices()[-1])  # [19:15] Player 3 declines to sell shares
g.process_action(
    get_all_choices()[1]
)  # [19:15] Player 4 buys a 10% share of CPR from the IPO for $100
g.process_action(get_all_choices()[-1])  # [19:15] Player 4 declines to sell shares
g.process_action(
    get_all_choices()[0]
)  # [19:15] Player 1 buys a 10% share of B&M from the IPO for $100
g.process_action(get_all_choices()[-1])  # [19:16] Player 1 declines to sell shares
g.process_action(
    get_all_choices()[1]
)  # [19:16] Player 2 buys a 10% share of CPR from the IPO for $100
# [19:16] CPR floats
# [19:16] CPR receives $1000
g.process_action(get_all_choices()[-1])  # [19:16] Player 2 declines to sell shares
g.process_action(
    get_all_choices()[2]
)  # [19:16] Player 3 buys a 10% share of PRR from the market for $90
g.process_action(get_all_choices()[-1])  # [19:16] Player 3 declines to sell shares
g.process_action(get_all_choices()[-1])  # [19:16] Player 4 passes
g.process_action(
    get_all_choices()[0]
)  # [19:16] Player 1 buys a 10% share of B&M from the IPO for $100
g.process_action(get_all_choices()[-1])  # [19:16] Player 1 declines to sell shares
g.process_action(
    get_all_choices()[1]
)  # [19:16] Player 2 buys a 10% share of CPR from the IPO for $100
g.process_action(get_all_choices()[-1])  # [19:17] Player 2 declines to sell shares
g.process_action(
    get_all_choices()[1]
)  # [19:18] Player 3 buys a 10% share of CPR from the IPO for $100
g.process_action(get_all_choices()[-1])  # [19:18] Player 3 declines to sell shares
g.process_action(get_all_choices()[-1])  # [19:18] Player 4 passes
g.process_action(
    get_all_choices()[0]
)  # [19:18] Player 1 buys a 10% share of B&M from the IPO for $100
# [19:18] B&M floats
# [19:18] B&M receives $1000
g.process_action(get_all_choices()[-1])  # [19:18] Player 1 declines to sell shares
g.process_action(
    get_all_choices()[-4]
)  # [19:18] Player 2 sells 2 shares of NYNH and receives $200
# [19:18] NYNH's share price moves down from $100 to $80
g.process_action(
    get_all_choices()[1]
)  # [19:18] Player 2 buys a 10% share of CPR from the IPO for $100
g.process_action(get_all_choices()[-1])
g.process_action(get_all_choices()[-1])  # [19:19] Player 3 passes
g.process_action(get_all_choices()[-1])  # [19:19] Player 4 passes
g.process_action(get_all_choices()[-1])  # [19:19] Player 1 passes
g.process_action(
    get_all_choices()[1]
)  # [19:19] Player 2 buys a 10% share of CPR from the IPO for $100
g.process_action(get_all_choices()[-1])  # [19:19] Player 2 declines to sell shares
g.process_action(get_all_choices()[-1])  # [19:19] Player 3 passes
g.process_action(get_all_choices()[-1])  # [19:19] Player 4 passes
g.process_action(get_all_choices()[-1])  # [19:19] Player 1 passes
g.process_action(get_all_choices()[-1])  # [19:19] Player 2 passes

In [ ]:
print_summary()

## Operating Round 6

### 6.1

In [ ]:
# [19:19] Player 2 operates C&O
g.process_action(
    get_all_choices()[10]
)  # [19:21] C&O lays tile #9 with rotation 1 on G9
# [19:21] C&O skips place a token
g.process_action(
    get_all_choices()[0]
)  # [19:21] C&O runs a 5 train for $220: E11-F16-G17-G19-H18
# [19:21] C&O runs a 3 train for $110: F2-F6-G7
g.process_action(
    get_all_choices()[0]
)  # [19:21] C&O pays out $330 = $33 per share ($198 to Player 2, $99 to Player 1, $33 to Player 4)
# [19:21] C&O's share price moves right from $100 to $111
# [19:21] C&O skips buy trains
# [19:21] C&O skips buy companies


# [19:21] Player 4 operates B&O
g.process_action(
    get_all_choices()[28]
)  # [19:22] B&O lays tile #46 with rotation 5 on G5
g.process_action(get_all_choices()[-1])  # [19:22] B&O passes place a token
g.process_action(
    get_all_choices()[0]
)  # [19:22] B&O runs a 4 train for $170: K13-J14-I15-H18
# [19:22] B&O runs a 3 train for $160: I15-H10-F2
g.process_action(get_all_choices()[1])  # [19:22] B&O withholds $330
# [19:22] B&O's share price moves left from $100 to $90
# [19:22] B&O skips buy trains
# [19:22] B&O skips buy companies


# [19:22] Player 1 operates B&M
# [19:22] B&M places a token on E23
g.process_action(
    get_all_choices()[11]
)  # [19:24] B&M lays tile #45 with rotation 4 on E21
g.process_action(get_all_choices()[-1])  # [19:24] B&M passes place a token
# [19:24] B&M skips run routes
# [19:24] B&M does not run
# [19:24] B&M's share price moves left from $100 to $90
g.process_action(
    get_all_choices()[0]
)  # [19:24] B&M buys a 5 train for $450 from The Depot
g.process_action(get_all_choices()[-1])  # [19:24] B&M passes buy trains
# [19:24] B&M skips buy companies


# [19:24] Player 2 operates CPR
# [19:24] CPR places a token on A19
g.process_action(
    get_all_choices()[-3]
)  # [19:24] CPR spends $80 and lays tile #9 with rotation 0 on B18
# [19:24] CPR skips place a token
# [19:24] CPR skips run routes
# [19:24] CPR does not run
# [19:24] CPR's share price moves left from $100 to $90
g.process_action(
    get_all_choices()[0]
)  # [19:24] CPR buys a 6 train for $630 from The Depot
# [19:24] -- Phase 6 (Operating Rounds: 3 | Train Limit: 2 | Available Tiles: Yellow, Green, Brown) --
# [19:24] -- Event: 3 trains rust ( C&O x1, NYC x2, NYNH x1, B&O x1) --
g.process_action(get_all_choices()[-1])  # [19:25] CPR passes buy trains
# [19:25] CPR skips buy companies


# [19:25] Player 2 operates ERIE
g.process_action(
    get_all_choices()[6]
)  # [19:25] ERIE lays tile #25 with rotation 2 on F12
# [19:25] ERIE skips place a token
g.process_action(get_all_choices()[0])  # [19:25] ERIE runs a 5 train for $80: E11-H10
g.process_action(get_all_choices()[1])  # [19:25] ERIE withholds $80
# [19:25] ERIE's share price moves left from $100 to $90
# [19:25] ERIE skips buy trains
# [19:25] ERIE skips buy companies


# [19:25] Player 4 operates PRR
g.process_action(
    get_all_choices()[8]
)  # [19:26] PRR lays tile #23 with rotation 1 on H14
# [19:26] PRR skips place a token
g.process_action(
    get_all_choices()[0]
)  # [19:26] PRR runs a 4 train for $170: F2-H10-H12-I15
g.process_action(get_all_choices()[1])  # [19:27] PRR withholds $170
# [19:27] PRR's share price moves left from $90 to $82
g.process_action(get_all_choices()[-1])  # [19:27] PRR passes buy trains
# [19:27] PRR skips buy companies


# [19:27] Player 3 operates NYNH
g.process_action(
    get_all_choices()[20]
)  # [19:27] NYNH lays tile #8 with rotation 4 on D12
# [19:27] NYNH skips place a token
g.process_action(
    get_all_choices()[0]
)  # [19:27] NYNH runs a 4 train for $190: G19-E19-D14-E11
g.process_action(get_all_choices()[1])  # [19:27] NYNH withholds $190
# [19:27] NYNH's share price moves left from $80 to $75
# [19:27] NYNH passes buy trains
# [19:27] NYNH skips buy companies


# [19:27] Player 1 operates NYC
g.process_action(
    get_all_choices()[-10]
)  # [19:27] NYC lays tile #40 with rotation 0 on F12
g.process_action(get_all_choices()[-1])  # [19:27] NYC passes place a token
# [19:27] NYC skips run routes
# [19:27] NYC does not run
# [19:27] NYC's share price moves left from $65 to $58
g.process_action(
    get_all_choices()[0]
)  # [19:27] NYC buys a 6 train for $630 from The Depot
g.process_action(get_all_choices()[-1])  # [19:28] NYC passes buy trains
# [19:28] NYC skips buy companies

In [ ]:
print_summary()

### 6.2

In [ ]:
# [19:28] Player 2 operates C&O
g.process_action(
    get_all_choices()[24]
)  # [20:27] C&O lays tile #23 with rotation 3 on G11
# [20:27] C&O skips place a token
g.process_action(
    get_all_choices()[0]
)  # [20:27] C&O runs a 5 train for $220: E11-F16-G17-G19-H18
g.process_action(
    get_all_choices()[0]
)  # [20:27] C&O pays out $220 = $22 per share ($132 to Player 2, $66 to Player 1, $22 to Player 4)
# [20:27] C&O's share price moves right from $111 to $125
g.process_action(get_all_choices()[-1])  # [20:27] C&O passes buy trains
# [20:27] C&O skips buy companies


# [20:27] Player 4 operates B&O
g.process_action(
    get_all_choices()[42]
)  # [20:27] B&O lays tile #43 with rotation 0 on G11
g.process_action(get_all_choices()[-1])  # [20:28] B&O passes place a token
g.process_action(
    get_all_choices()[0]
)  # [20:28] B&O runs a 4 train for $210: F2-H10-I15-H18
g.process_action(get_all_choices()[1])  # [20:28] B&O withholds $210
# [20:28] B&O's share price moves left from $90 to $82
g.process_action(
    get_all_choices()[-2]
)  # [20:28] B&O exchanges a 4 for a D train for $800 from The Depot
# [20:28] -- Phase D (Operating Rounds: 3 | Train Limit: 2 | Available Tiles: Yellow, Green, Brown) --
# [20:28] -- Event: 4 trains rust ( The Depot x1, PRR x1, NYNH x1, ERIE x1) --
# [20:28] B&O passes buy trains
# [20:28] B&O skips buy companies


# [20:28] Player 1 operates B&M
g.process_action(
    get_all_choices()[11]
)  # [20:28] B&M lays tile #8 with rotation 5 on D20
g.process_action(get_all_choices()[-1])  # [20:28] B&M passes place a token
g.process_action(
    get_all_choices()[0]
)  # [20:28] B&M runs a 5 train for $170: E23-F24-F22-F20-G19
g.process_action(
    get_all_choices()[0]
)  # [20:28] B&M pays out $170 = $17 per share ($102 to Player 1)
# [20:28] B&M's share price moves right from $90 to $100
g.process_action(get_all_choices()[-1])  # [20:28] B&M passes buy trains
# [20:28] B&M skips buy companies


# [20:28] Player 2 operates CPR
g.process_action(
    get_all_choices()[-3]
)  # [20:28] CPR spends $120 and lays tile #9 with rotation 0 on C17
# [20:28] CPR skips place a token
g.process_action(get_all_choices()[0])  # [20:29] CPR runs a 6 train for $50: A19-B20
g.process_action(
    get_all_choices()[0]
)  # [20:29] CPR pays out $50 = $5 per share ($30 to Player 2, $10 to Player 3, $10 to Player 4)
# [20:29] CPR's share price moves right from $90 to $100
g.process_action(get_all_choices()[-1])  # [20:29] CPR passes buy trains
# [20:29] CPR skips buy companies


# [20:29] Player 2 operates ERIE
g.process_action(
    get_all_choices()[0]
)  # [20:29] ERIE spends $80 and lays tile #59 with rotation 3 on D10 (Hamilton & Toronto)
# [20:29] ERIE skips place a token
g.process_action(
    get_all_choices()[0]
)  # [20:29] ERIE runs a 5 train for $130: D10-E11-G7-F6
g.process_action(
    get_all_choices()[0]
)  # [20:29] ERIE pays out $130 = $13 per share ($65 to Player 2, $52 to Player 4, $13 to Player 3)
# [20:29] ERIE's share price moves right from $90 to $100
# [20:29] ERIE skips buy trains
# [20:29] ERIE skips buy companies


# [20:29] Player 4 operates PRR
g.process_action(
    get_all_choices()[11]
)  # [20:29] PRR lays tile #57 with rotation 1 on H16 (Lancaster)
g.process_action(get_all_choices()[-1])  # [20:29] PRR passes place a token
# [20:29] PRR skips run routes
# [20:29] PRR does not run
# [20:29] PRR's share price moves left from $82 to $76
g.process_action(
    get_all_choices()[440]
)  # [20:30] Player 4 sells 4 shares of ERIE and receives $400
# [20:30] ERIE's share price moves down from $100 to $71
g.process_action(
    get_all_choices()[433]
)  # [20:30] Player 4 sells a 10% share of CPR and receives $100
# [20:30] CPR's share price moves down from $100 to $90
g.process_action(get_all_choices()[0])  # [20:30] Player 4 contributes $670
# [20:30] PRR buys a D train for $1100 from The Depot
# [20:30] PRR skips buy companies


# [20:30] Player 3 operates NYNH
g.process_action(
    get_all_choices()[37]
)  # [20:30] NYNH lays tile #58 with rotation 3 on F10 (Erie)
# [20:30] NYNH skips place a token
# [20:30] NYNH skips run routes
# [20:30] NYNH does not run
# [20:30] NYNH's share price moves left from $75 to $71
g.process_action(
    get_all_choices()[9]
)  # [20:31] Player 3 sells 4 shares of B&O and receives $328
# [20:31] B&O's share price moves down from $82 to $62
g.process_action(
    get_all_choices()[5]
)  # [20:31] Player 3 sells 2 shares of CPR and receives $180
# [20:31] CPR's share price moves down from $90 to $76
g.process_action(
    get_all_choices()[4]
)  # [20:31] Player 3 sells a 10% share of ERIE and receives $71
# [20:31] ERIE's share price moves down from $71 to $67
g.process_action(
    get_all_choices()[3]
)  # [20:31] Player 3 sells 3 shares of NYC and receives $174
# [20:31] NYC's share price moves down from $58 to $40
g.process_action(
    get_all_choices()[0]
)  # [20:31] Player 3 sells a 10% share of PRR and receives $76
# [20:31] PRR's share price moves down from $76 to $71
g.process_action(get_all_choices()[0])  # [20:31] Player 3 contributes $860
# [20:31] NYNH buys a D train for $1100 from The Depot
# [20:31] NYNH skips buy companies


# [20:31] Player 1 operates NYC
g.process_action(
    get_all_choices()[56]
)  # [20:31] NYC lays tile #24 with rotation 4 on G9
g.process_action(
    get_all_choices()[3]
)  # [20:32] NYC places a token on E11 (Dunkirk & Buffalo) for $100
g.process_action(
    get_all_choices()[0]
)  # [20:32] NYC runs a 6 train for $280: I15-H18-G19-G17-F16-E11
g.process_action(
    get_all_choices()[0]
)  # [20:32] NYC pays out $280 = $28 per share ($140 to Player 1, $84 to NYC, $56 to Player 4)
# [20:32] NYC's share price moves right from $40 to $50
g.process_action(get_all_choices()[-1])  # [20:32] NYC passes buy trains
# [20:32] NYC skips buy companies

In [ ]:
print_summary()

### 6.3

In [ ]:
# [20:32] Player 2 operates C&O
g.process_action(
    get_all_choices()[52]
)  # [21:35] C&O lays tile #24 with rotation 1 on H8
# [21:35] C&O skips place a token
router.debug = True
g.process_action(
    get_all_choices()[0]
)  # [21:36] C&O runs a 5 train for $240: F2-F16-G17-G19-H18
g.process_action(
    get_all_choices()[0]
)  # [21:36] C&O pays out $240 = $24 per share ($144 to Player 2, $72 to Player 1, $24 to Player 4)
# [21:36] C&O's share price moves right from $125 to $140
g.process_action(get_all_choices()[-1])  # [21:36] C&O passes buy trains
# [21:36] C&O skips buy companies


# [21:36] Player 1 operates B&M
g.process_action(
    get_all_choices()[17]
)  # [21:36] B&M lays tile #23 with rotation 1 on D18
g.process_action(
    get_all_choices()[-1]
)  # [21:36] B&M places a token on D14 (Rochester) for $40
g.process_action(
    get_all_choices()[0]
)  # [21:36] B&M runs a 5 train for $170: E23-F24-F22-F20-G19
g.process_action(
    get_all_choices()[0]
)  # [21:36] B&M pays out $170 = $17 per share ($102 to Player 1)
# [21:36] B&M's share price moves right from $100 to $112
g.process_action(get_all_choices()[-1])  # [21:36] B&M passes buy trains
# [21:36] B&M skips buy companies


# [21:36] Player 2 operates CPR
g.process_action(
    get_all_choices()[27]
)  # [21:36] CPR lays tile #20 with rotation 0 on D16
# [21:36] CPR skips place a token
g.process_action(get_all_choices()[0])  # [21:36] CPR runs a 6 train for $50: A19-B20
g.process_action(
    get_all_choices()[0]
)  # [21:36] CPR pays out $50 = $5 per share ($30 to Player 2, $15 to CPR, $5 to Player 4)
# [21:36] CPR's share price moves right from $76 to $82
g.process_action(get_all_choices()[-1])  # [21:36] CPR passes buy trains
# [21:36] CPR skips buy companies


# [21:36] Player 3 operates NYNH
g.process_action(
    get_all_choices()[9]
)  # [21:37] NYNH lays tile #47 with rotation 0 on D16
# [21:37] NYNH skips place a token
g.process_action(
    get_all_choices()[0]
)  # [21:37] NYNH runs a D train for $250: E23-F24-F22-F20-G19-E19-F20-F16
g.process_action(
    get_all_choices()[0]
)  # [21:37] NYNH pays out $250 = $25 per share ($150 to Player 3, $50 to Player 2, $50 to NYNH)
# [21:37] NYNH's share price moves right from $71 to $75
# [21:37] NYNH passes buy trains
# [21:37] NYNH skips buy companies


# [21:37] Player 4 operates PRR
g.process_action(
    get_all_choices()[35]
)  # [21:37] PRR lays tile #24 with rotation 1 on F14
# [21:37] PRR skips place a token
g.process_action(
    get_all_choices()[0]
)  # [21:37] PRR runs a D train for $180: F2-H10-H12-H16-H18
g.process_action(
    get_all_choices()[0]
)  # [21:37] PRR pays out $180 = $18 per share ($72 to Player 4, $54 to Player 1, $54 to PRR)
# [21:37] PRR's share price moves right from $71 to $76
g.process_action(get_all_choices()[-1])  # [21:37] PRR passes buy trains
# [21:37] PRR skips buy companies


# [21:37] Player 2 operates ERIE
g.process_action(
    get_all_choices()[7]
)  # [21:38] ERIE lays tile #67 with rotation 3 on D10 (Hamilton & Toronto)
# [21:38] ERIE skips place a token
g.process_action(
    get_all_choices()[0]
)  # [21:38] ERIE runs a 5 train for $170: F2-E11-D10
g.process_action(
    get_all_choices()[0]
)  # [21:38] ERIE pays out $170 = $17 per share ($85 to Player 2, $85 to ERIE)
# [21:38] ERIE's share price moves right from $67 to $71
g.process_action(get_all_choices()[-1])  # [21:38] ERIE passes buy trains
# [21:38] ERIE skips buy companies


# [21:38] Player 4 operates B&O
g.process_action(
    get_all_choices()[36]
)  # [21:38] B&O lays tile #9 with rotation 0 on E15
g.process_action(get_all_choices()[-1])  # [21:39] B&O passes place a token
g.process_action(
    get_all_choices()[0]
)  # [21:39] B&O runs a D train for $220: F2-H10-I15-J14-K13
g.process_action(
    get_all_choices()[0]
)  # [21:39] B&O pays out $220 = $22 per share ($110 to Player 4, $88 to B&O, $22 to Player 1)
# [21:39] B&O's share price moves right from $62 to $67
g.process_action(get_all_choices()[-1])  # [21:39] B&O passes buy trains
# [21:39] B&O skips buy companies


# [21:39] Player 1 operates NYC
g.process_action(
    get_all_choices()[20]
)  # [21:39] NYC lays tile #28 with rotation 0 on D12
# [21:39] NYC skips place a token
g.process_action(
    get_all_choices()[0]
)  # [21:39] NYC runs a 6 train for $300: F2-F16-G17-G19-H18-I15
g.process_action(
    get_all_choices()[0]
)  # [21:39] NYC pays out $300 = $30 per share ($150 to Player 1, $90 to NYC, $60 to Player 4)
# [21:39] NYC's share price moves right from $50 to $60
g.process_action(get_all_choices()[-1])  # [21:39] NYC passes buy trains
# [21:39] NYC skips buy companies

In [ ]:
print_summary()

## Stock Round 7

In [ ]:
g.process_action(
    get_all_choices()[5]
)  # [21:46] Player 3 buys a 10% share of NYC from the market for $60
g.process_action(get_all_choices()[-1])  # [21:46] Player 3 declines to sell shares

g.process_action(
    get_all_choices()[6]
)  # [21:46] Player 4 buys a 10% share of NYC from the market for $60
g.process_action(get_all_choices()[-1])  # [21:47] Player 4 declines to sell shares

g.process_action(
    get_all_choices()[5]
)  # [21:47] Player 1 buys a 10% share of NYC from the market for $60
g.process_action(get_all_choices()[-1])  # [21:47] Player 1 declines to sell shares

g.process_action(get_all_choices()[-1])  # [21:47] Player 2 passes

g.process_action(
    get_all_choices()[4]
)  # [21:47] Player 3 buys a 10% share of B&O from the market for $67
g.process_action(get_all_choices()[-1])  # [21:47] Player 3 declines to sell shares

g.process_action(
    get_all_choices()[5]
)  # [21:47] Player 4 buys a 10% share of B&O from the market for $67
g.process_action(get_all_choices()[-1])  # [21:47] Player 4 declines to sell shares

g.process_action(
    get_all_choices()[4]
)  # [21:47] Player 1 buys a 10% share of B&O from the market for $67
g.process_action(get_all_choices()[-1])  # [21:47] Player 1 declines to sell shares

g.process_action(get_all_choices()[-1])  # [21:48] Player 2 passes

g.process_action(get_all_choices()[-1])  # [21:48] Player 3 passes

g.process_action(
    get_all_choices()[2]
)  # [21:48] Player 4 buys a 10% share of PRR from the market for $76
g.process_action(get_all_choices()[-1])  # [21:48] Player 4 declines to sell shares

# [21:48] Player 1 buys a 10% share of B&O from the market for $67
g.process_action(get_all_choices()[-1])  # [21:48] Player 1 declines to sell shares

g.process_action(get_all_choices()[-1])  # [21:48] Player 2 passes

g.process_action(get_all_choices()[-1])  # [21:48] Player 3 passes

# [21:48] Player 4 buys a 10% share of PRR from the market for $76
g.process_action(get_all_choices()[-1])  # [21:48] Player 4 declines to sell shares

# [21:48] Player 1 buys a 10% share of ERIE from the market for $71
g.process_action(get_all_choices()[-1])  # [21:48] Player 1 declines to sell shares

g.process_action(get_all_choices()[-1])  # [21:48] Player 2 passes

g.process_action(get_all_choices()[-1])  # [21:48] Player 3 passes

g.process_action(get_all_choices()[-1])  # [21:48] Player 4 passes

# [21:49] Player 1 buys a 10% share of ERIE from the market for $71
g.process_action(get_all_choices()[-1])  # [21:49] Player 1 declines to sell shares

g.process_action(get_all_choices()[-1])  # [21:49] Player 2 passes

g.process_action(get_all_choices()[-1])  # [21:49] Player 3 passes

g.process_action(get_all_choices()[-1])  # [21:49] Player 4 passes

g.process_action(get_all_choices()[-1])  # [21:49] Player 1 passes

## WIP

In [ ]:
# g.process_action(get_all_choices()[2])

In [ ]:
# g.process_action(get_all_choices()[-1])

In [ ]:
g.current_entity.shares[1].corporation().counts_for_limit()

In [ ]:
print_summary()

In [ ]:
# g.debug = True
# router.debug = True
print(
    f"step: {g.active_step()}, player: {g.current_entity}, purchasing power: {g.buying_power(g.current_entity) if hasattr(g.current_entity, 'cash') else 0}"
)
print(f"possible actions: {get_current_actions()}")
print(f"company actions: {get_company_actions()}")
get_all_choices_with_index()

In [ ]:
from rl18xx.game.engine.graph import copy_dict

d = {"a": ["a", "b", "c"]}

d2 = copy_dict(d)

print(d)
print(d2)
d["a"].append("d")
print(d)
print(d2)

## TODO
* Debug auto-router
* * Need to keep individual visited history per node and not stop looking when you get to a path you've used already
* Finish game

In [ ]:
g.log